In [1]:
# https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0#:~:text=In%20this%20tutorial%20we%20will%20implement%20a%20simple,allow%20you%20to%20build%20your%20customized%20neural%20networks.

# we transform a simple model archtecture and use bit encoding

In [2]:
import torch
import torch.nn as nn

In [3]:
# sample data
X = torch.tensor(([2, 9], [1, 5], [3, 6]), dtype=torch.float) # 3 X 2 tensor
y = torch.tensor(([92], [100], [89]), dtype=torch.float) # 3 X 1 tensor
xPredicted = torch.tensor(([4, 8]), dtype=torch.float) # 1 X 2 tensor

In [4]:
print(X.size())
print(y.size())

torch.Size([3, 2])
torch.Size([3, 1])


In [5]:
# scale units
X_max, _ = torch.max(X, 0)
xPredicted_max, _ = torch.max(xPredicted, 0)

X = torch.div(X, X_max)
xPredicted = torch.div(xPredicted, xPredicted_max)
y = y / 100  # max test score is 100

In [6]:
# import torch
# T_data = [[[1., 2., 3.], [4., 5., 6.]], [[7., 8., 9.], [10., 11., 12.]], [[13., 14., 15.], [16., 17., 18.]]]
# T = torch.tensor(T_data, dtype=torch.float)
# print(T.shape)

In [7]:
X_mean = torch.mean(X, dim=0, keepdim=True)
X_std = torch.std(X, dim=0, keepdim=True)
X_normalized = (X - X_mean) / X_std


In [8]:
xPredicted_normalized = (xPredicted - X_mean) / X_std


In [9]:
class Neural_Network(nn.Module):
    def __init__(self, learning_rate=0.001):
        super(Neural_Network, self).__init__()
        # parameters
        self.inputSize = 2
        self.outputSize = 1
        self.hiddenSize = 3
        self.learning_rate = learning_rate

        self.relu = nn.ReLU()

        # weights
        self.W1 = torch.randn(self.inputSize, self.hiddenSize) # 2 X 3 tensor
        self.W2 = torch.randn(self.hiddenSize, self.outputSize) # 3 X 1 tensor
        self.bias1 = torch.randn(self.hiddenSize) # 3 tensor
        self.bias2 = torch.randn(self.outputSize) # 1 tensor

    def forward(self, X):
        self.z = torch.matmul(X, self.W1) + self.bias1 # broadcast addition
        self.z2 = self.leaky_relu(self.z) # activation function
        self.z3 = torch.matmul(self.z2, self.W2) + self.bias2 # broadcast addition
        o = self.leaky_relu(self.z3) # final activation function
        return o

    # def sigmoid(self, s):
    #     return 1 / (1 + torch.exp(-s))

    # def sigmoidPrime(self, s):
    #     return s * (1 - s)

    def leaky_relu(self, s, negative_slope=0.01):
      return torch.where(s >= 0, s, s * negative_slope)

    def leaky_relu_prime(self, s, negative_slope=0.01):
      return torch.where(s >= 0, torch.ones_like(s), torch.full_like(s, negative_slope))


    def backward(self, X, y, o):
        self.o_error = y - o # error in output
        self.o_delta = self.o_error * self.leaky_relu_prime(self.z3) # applying derivative of sigmoid to error
        self.z2_error = torch.matmul(self.o_delta, torch.t(self.W2))
        self.z2_delta = self.z2_error * self.leaky_relu_prime(self.z2)

        dW1 = torch.matmul(torch.t(X), self.z2_delta)
        dW2 = torch.matmul(torch.t(self.z2), self.o_delta)

        dbias1 = torch.sum(self.z2_delta, dim=0)
        dbias2 = torch.sum(self.o_delta, dim=0)

        # update weights and biases with scaled gradients
        self.W1 += dW1 * self.learning_rate
        self.W2 += dW2 * self.learning_rate
        self.bias1 += dbias1 * self.learning_rate
        self.bias2 += dbias2 * self.learning_rate

    def train(self, X, y):
        # forward + backward pass for training
        o = self.forward(X)
        self.backward(X, y, o)

    def saveWeights(self, model):
        # we will use the PyTorch internal storage functions
        torch.save(model, "NN")

    def predict(self):
        print ("Predicted data based on trained weights: ")
        print ("Input (scaled): \n" + str(xPredicted))
        print ("Output: \n" + str(self.forward(xPredicted)))


In [ ]:
# Training
# loss begins to increase!
NN = Neural_Network()
for i in range(1000):  # trains the NN 10,000 times
    print ("#" + str(i) + " Loss: " + str(torch.mean((y - NN(X_normalized))**2).detach().item()))  # mean sum squared loss
    NN.train(X_normalized, y)
NN.saveWeights(NN)

# Predict using normalized data
print ("Predicted data based on trained weights: ")
print ("Input (scaled): \n" + str(xPredicted_normalized))
print ("Output: \n" + str(NN(xPredicted_normalized)))

In [12]:
# we apply early stopping
NN = Neural_Network()
min_val_loss = float('inf')
patience = 10  # Number of epochs to wait before stopping training if validation loss does not decrease
wait = 0

for i in range(1000):
    # Training
    train_loss = torch.mean((y - NN(X_normalized))**2).detach().item()
    print("#" + str(i) + " Train Loss: " + str(train_loss))
    NN.train(X_normalized, y)

    # Validation
    # val_loss = ...  # Calculate validation loss here
    # print("#" + str(i) + " Validation Loss: " + str(val_loss))

    # Early stopping
    if train_loss < min_val_loss:
        min_val_loss = train_loss
        wait = 0
        # Save the best model weights here
        NN.saveWeights(NN)
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping")
            break

# Load the best model weights here
# NN.loadWeights()

# Predict using normalized data
print("Predicted data based on trained weights: ")
print("Input (scaled): \n" + str(xPredicted_normalized))
print("Output: \n" + str(NN(xPredicted_normalized)))


#0 Train Loss: 0.9073054790496826
#1 Train Loss: 0.9073049426078796
#2 Train Loss: 0.9073043465614319
#3 Train Loss: 0.9073037505149841
#4 Train Loss: 0.9073031544685364
#5 Train Loss: 0.9073026180267334
#6 Train Loss: 0.9073019027709961
#7 Train Loss: 0.9073013663291931
#8 Train Loss: 0.9073007106781006
#9 Train Loss: 0.9073001742362976
#10 Train Loss: 0.9072995781898499
#11 Train Loss: 0.9072989821434021
#12 Train Loss: 0.9072983860969543
#13 Train Loss: 0.907297670841217
#14 Train Loss: 0.9072971940040588
#15 Train Loss: 0.9072966575622559
#16 Train Loss: 0.9072960019111633
#17 Train Loss: 0.9072954058647156
#18 Train Loss: 0.9072949290275574
#19 Train Loss: 0.9072942733764648
#20 Train Loss: 0.9072936177253723
#21 Train Loss: 0.9072930216789246
#22 Train Loss: 0.907292366027832
#23 Train Loss: 0.9072918891906738
#24 Train Loss: 0.9072912335395813
#25 Train Loss: 0.9072906374931335
#26 Train Loss: 0.9072900414466858
#27 Train Loss: 0.907289445400238
#28 Train Loss: 0.907288849353790